In [1]:
import pandas as pd
import pandas_datareader.data as web
from pandas import Series, DataFrame
import datetime
from datetime import date, timedelta, datetime
import math
import matplotlib as mpl
import numpy as np
import talib as ta
import yfinance as yf
from tqdm import tqdm
from os import walk
import os
import matplotlib.pyplot as plt
import random
from random import randrange


In [2]:
def random_date(start, end): #This is to generate random dates
    delta = end - start
    int_delta = (delta.days * 24 * 60 * 60) + delta.seconds
    random_second = randrange(int_delta)
    return start + timedelta(seconds=random_second)

In [3]:
#This is for version 2(numpy)
def buyStock2(row):
    close_adj = row[4]
    starting_price = close_adj
    return close_adj,starting_price,True

def sellStock2(row,starting_price):
    close_adj = row[4]
    single_trade_percent_gain = ((close_adj - starting_price) / starting_price) 
    return close_adj,single_trade_percent_gain,False

In [4]:
def tradeVerison2(stock, startDate, endDate,plotVal):
    method1 = 0
    method2 = 0
    StartingAmount = 500_000/505
    oldDate = startDate
    startDate = startDate - timedelta(39)
    df = pd.read_csv(
        "C:/Users/rohan/Desktop/SchoolWork/StockMarketSimulationIQP/RohanCode/Data/filter_30y_stock_csvs/{}.csv".format(stock))
    df = df.rename(columns={"CLOSE": "Adj Close", "VOLUME": "Volume"})

    df['Datetime'] = pd.to_datetime(df['Datetime'])
    g_fd = df
    g_fd = g_fd[~(g_fd['Datetime'] < oldDate)]
    g_fd = g_fd[~(g_fd['Datetime'] > endDate)]
  
    df = df[~(df['Datetime'] < startDate)]
    df = df[~(df['Datetime'] > endDate)]

    #Make Data Frame only show the right dates


    StockData = pd.DataFrame(df).to_numpy() #Convert df to a array
    
    mapForBuy = {}
    WindowVal = 11 #Opt number for window size
    mapForSellP = {}
    mapForSellN = {}

    gainsList = []
    lostList = []

    Total_Gain = 0
    starting_price = 1

    TotalReturn = StartingAmount
    index = 28 #Start on index 28 for the right date
    bought = False
    val = StockData.shape
    for rows in StockData:
        row = StockData[index][:]
        if(index >= val[0] -1):
            break

        # maxValuePeriod = df.iloc[index-7:index-1]['Adj Close'].max()

        # Valuemean = df.iloc[index-7:index-1]['Adj Close'].mean()
        # ValuemeanOld = df.iloc[index-14:index-7]['Adj Close'].mean()
        # stvalue = df.iloc[index-10:index-1]['Adj Close'].std()

        maxValuePeriod = df.iloc[index-WindowVal:index-1]['Adj Close'].max()

        Valuemean = df.iloc[index-WindowVal:index-1]['Adj Close'].mean()

        ValuemeanOld = df.iloc[index-(WindowVal+7):index-7]['Adj Close'].mean()
        stvalue = df.iloc[index-(WindowVal+3):index-1]['Adj Close'].std()

        if math.isnan(maxValuePeriod):
            maxValuePeriod = 0
        if math.isnan(Valuemean):
            Valuemean = 0

        if not bought:
            if(StockData[index-1][8] > 11):

                if StockData[index-1][11] == 1 and StockData[index-1][10] == 1:
                    close_adj, starting_price, bought = buyStock2(row)
                    mapForBuy[row[0]] = row[4]
                    method1+=1
                
                elif maxValuePeriod - stvalue  < Valuemean:
                        
                    close_adj, starting_price, bought = buyStock2(row)
                    mapForBuy[row[0]] = row[4]
                    method2+=1
                    
                else:
                    gainsList.append(0)

        elif bought and close_adj != row[4]:  # This is sell clause
            if (StockData[index-1][11] == 0 or StockData[index-1][9] < 41) and StockData[index-1][10] == 0 and ValuemeanOld < Valuemean:
                infoOnSelling = sellStock2(row, starting_price)
                close_adj = infoOnSelling[0]
                Total_Gain += infoOnSelling[1]
                bought = infoOnSelling[2]

                gainsList.append(infoOnSelling[1])

                if(infoOnSelling[1] >= 0):
                    mapForSellP[row[0]] = row[4]

                else:
                    lostList.append(infoOnSelling[1])
                    mapForSellN[row[0]] = row[4]

            else:
                yesterday = StockData[index-1][4]
                today = row[4]
                checker = (today - yesterday)/yesterday#Checks to see if money has been lost vs gained
                if(checker < 0):
                    lostList.append(checker)
                gainsList.append(checker)
                
        index += 1
        TotalReturn = (1 + Total_Gain) * TotalReturn
        Total_Gain = 0

    if bought:#Last day of simulation
        row = StockData[index][:]
        infoOnSelling = sellStock2(row, starting_price)
        close_adj = infoOnSelling[0]
        Total_Gain += infoOnSelling[1]
        bought = infoOnSelling[2]

        gainsList.append(infoOnSelling[1])

        if(infoOnSelling[1] >= 0):
            mapForSellP[row[0]] = row[4]

        else:
            lostList.append(infoOnSelling[1])
            mapForSellN[row[0]] = row[4]

        TotalReturn = (1 + Total_Gain) * TotalReturn
        Total_Gain = 0

    if(plotVal):#Plot graphs
        plt.plot(g_fd['Datetime'], g_fd['Adj Close'])
        plt.scatter(mapForBuy.keys(), mapForBuy.values(), label="Buy", color="Blue",
                    marker="o", s=30)
        plt.scatter(mapForSellP.keys(), mapForSellP.values(), label="Sell - Gain", color="Green",
                    marker="^", s=30)
        plt.scatter(mapForSellN.keys(), mapForSellN.values(), label="Sell - Lost", color="Red",
                    marker="v", s=30)

        plt.xlabel("Date")
        plt.ylabel("Price $")
        plt.title("{}".format(stock))
        plt.tick_params(axis='x',which='major', labelsize=7, rotation = 90)
        #plt.tick_params(axis='y',colors='white')
        
        plt.legend()
        plt.tight_layout()

        plt.savefig('GraphsForBuySell/{}.png'.format(stock),facecolor = 'w')
        plt.close()

    tradingdays = (endDate-startDate).days
    if not mapForBuy or not np.std(gainsList):
        sharpeRatio = 0
    else:
        sharpeRatio = np.mean(gainsList)/np.std(gainsList)
        sharpeRatio = (tradingdays ** 0.5) * sharpeRatio

    if not lostList or not np.std(lostList):
        sortinoRatio = 0
    else:
        sortinoRatio = (np.mean(gainsList)/np.std(lostList)) * (tradingdays ** 0.5)
    
    return round(TotalReturn-StartingAmount, 2), sharpeRatio, sortinoRatio, method1,method2


In [9]:
f = []
money = {}
startDate = datetime.strptime('2021-03-19',"%Y-%m-%d")
offsetDate = timedelta(39)
endDate = datetime.strptime('2021-05-19',"%Y-%m-%d")
sum = 0
method1 =0 
method2 = 0
sharpeRatioAll = 0
sortinoRatioAll = 0
for (dirpath, dirnames, filenames) in walk("C:/Users/rohan/Desktop/SchoolWork/StockMarketSimulationIQP/RohanCode/Data/clean_30y_stock_csvs/"):
    f.extend(filenames)#Grab all file names
    break

#stock = 'TSLA'
for stock in tqdm(f):
    stock = stock[:-4]
    profit, sharpeRatio, sortinoRatio,m1,m2 = tradeVerison2(stock,startDate,endDate,False)
    money[stock] = profit
    sharpeRatioAll += sharpeRatio
    sortinoRatioAll += sortinoRatio
    method1 += m1
    method2 +=m2
for s in money:
    sum+= money[s]
sharpeRatioAll /= len(f)
sortinoRatioAll /= len(f)
percentageOfWin = ((sum)/500_000) * 100
print()
print("Total win of ${} with % of {}%".format(round(sum,2),round(percentageOfWin,2)))
print("Sharpe Ratio: {}".format(round(sharpeRatioAll,2)))
print("Sortino Ratio: {}".format(round(sortinoRatioAll,2)))
print("M1 was called: {} times and M2 was called {} times".format(method1,method2))

100%|██████████| 505/505 [00:29<00:00, 17.12it/s]


Total win of $20075.41 with % of 4.02%
Sharpe Ratio: 1.45
Sortino Ratio: 4.13
M1 was called: 376 times and M2 was called 1243 times


In [9]:
sp500 = yf.download('^GSPC',startDate,endDate)
profitsp500percentage = ((sp500['Close'][-1] - sp500['Close'][0])/sp500['Close'][0])*100
profitsp500 = (500_000) * (profitsp500percentage/100)

print("This is SAP 500 : {}% and the increase in money: ${}".format(round(profitsp500percentage,2),round(profitsp500,2)))

[*********************100%***********************]  1 of 1 completed
This is SAP 500 : 5.49% and the increase in money: $27437.32


In [6]:
totalWins = 0
f = []
testCases = 10
for (dirpath, dirnames, filenames) in walk("C:/Users/rohan/Desktop/SchoolWork/StockMarketSimulationIQP/RohanCode/Data/clean_30y_stock_csvs/"):
    f.extend(filenames)
    break

for i in range(testCases):

    StartDateRange = datetime.strptime('1/1/2000', '%m/%d/%Y')
    EndDateRange = datetime.strptime('10/1/2021', '%m/%d/%Y')
    
    startDate = random_date(StartDateRange, EndDateRange)
    startDate = startDate.strftime('%Y-%m-%d')
    startDate = datetime.strptime(startDate, "%Y-%m-%d")

    days = timedelta(61)
    endDate = startDate + days
    
    money = {}
    sum = 0
    sharpeRatioAll = 0
    sortinoRatioAll = 0

    for stock in f:
        stock = stock[:-4]
        profit, sharpeRatio, sortinoRatio,val1,val2 = tradeVerison2(stock,startDate,endDate,False)
        money[stock] = profit
        sharpeRatioAll += sharpeRatio
        sortinoRatioAll += sortinoRatio
    
    for s in money:
        sum+= money[s]
        sharpeRatioAll /= len(f)
        sortinoRatioAll /= len(f)
        percentageOfWin = ((sum)/500_000) * 100

    sp500 = yf.download('^GSPC',startDate,endDate)
    profitsp500percentage = ((sp500['Close'][-1] - sp500['Close'][0])/sp500['Close'][0])*100

    if percentageOfWin >= profitsp500percentage:
        totalWins += 1
    print("Done with loop number {} with dates of {} to {}. \nThe gain was {}% compared to SAP which was {}%".format(i+1,startDate,endDate,round(percentageOfWin,2),round(profitsp500percentage,2)))

win_per = (totalWins/testCases) * 100
win_per = round(win_per,2)
print("All wins in the test: {}% which is {} / {}".format(win_per,totalWins,testCases))

[*********************100%***********************]  1 of 1 completed
Done with loop number 1 with dates of 2012-09-20 00:00:00 to 2012-11-20 00:00:00. 
The gain was -0.77% compared to SAP which was -5.02%
[*********************100%***********************]  1 of 1 completed
Done with loop number 2 with dates of 2019-09-30 00:00:00 to 2019-11-30 00:00:00. 
The gain was 5.39% compared to SAP which was 5.52%
[*********************100%***********************]  1 of 1 completed
Done with loop number 3 with dates of 2004-09-04 00:00:00 to 2004-11-04 00:00:00. 
The gain was 3.39% compared to SAP which was 1.95%
